In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import json
import pickle

/Users/jare/Library/CloudStorage/GoogleDrive-johnallan.e@gmail.com/Other computers/My MacBook Pro/Applied AI Solutions/4011 - Math for DL 1/Project/.venv-inference/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load the trained Keras model from the .keras file
base_argument = "model_0_shakespeare/shakespeare"
model = load_model(f'{base_argument}_model.keras')

# Load the vocabulary mappings
with open(f'{base_argument}_word_to_id.json', 'r') as f:
    word_to_id = json.load(f)

# The id_to_word dictionary can be created from word_to_id
id_to_word = {v: k for k, v in word_to_id.items()}

# Load max_seq_length from pkl file
preprocessed_data = pickle.load(open(f'{base_argument}_preprocessed_data.pkl', 'rb'))
max_seq_length = preprocessed_data['max_seq_length']

2025-09-23 13:36:38.410762: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-09-23 13:36:38.410920: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-23 13:36:38.410926: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-23 13:36:38.410943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-23 13:36:38.410953: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def generate_text(model, seed_text, num_words_to_generate, word_to_id, id_to_word, max_seq_length):
    # Pre-process the seed text
    # It must be in the same format as your training data: lowercase and numerical
    processed_seed = [word_to_id.get(word.lower(), word_to_id['<UNK>']) for word in seed_text.split()]
    
    generated_text = seed_text
    
    for _ in range(num_words_to_generate):
        # Pad the input sequence to the required length
        padded_sequence = np.array(processed_seed[-max_seq_length:] + [word_to_id['<PAD>']] * (max_seq_length - len(processed_seed[-max_seq_length:])))
        
        # Reshape for the model's input layer
        padded_sequence = padded_sequence.reshape(1, max_seq_length)
        
        # Get the model's prediction
        # The output is a probability distribution over the entire vocabulary
        predictions = model.predict(padded_sequence, verbose=0)[0]
        
        # Sample the next word from the distribution
        # This is a key step to make the output more varied and creative
        predicted_id = np.random.choice(len(predictions), p=predictions)
        
        # Convert the integer ID back to a word
        predicted_word = id_to_word[predicted_id]
        
        # Stop generation if the end-of-sentence token is predicted
        if predicted_word == '<EOS>':
            break
            
        generated_text += " " + predicted_word
        processed_seed.append(predicted_id)

    return generated_text

In [4]:
# Example usage
seed_text = "Yesterday I saw a"

for s in range(10):
    generated_sentence = generate_text(model, seed_text, 50, word_to_id, id_to_word, max_seq_length)
    print(generated_sentence)


2025-09-23 13:36:39.801846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Yesterday I saw a bill
Yesterday I saw a book
Yesterday I saw a ship
Yesterday I saw a bill
Yesterday I saw a wit
Yesterday I saw a bill
Yesterday I saw a chain
Yesterday I saw a book
Yesterday I saw a book
Yesterday I saw a lap
